In [16]:
from openai import OpenAI
from pydantic import BaseModel
import sqlite3
from functools import wraps
from pydantic import Field, ConfigDict
import json
from typing import Dict, Any

In [3]:
con = sqlite3.connect("workoutlib.db")
cur = con.cursor()

In [12]:
def tool(schema):
    def decorator(func):
        assert issubclass(schema, BaseModel), "The schema must be an instance of Pydantic BaseModel"
        @wraps(func)
        def wrapper(*args, **kwargs):
            return func(*args, **kwargs)
        wrapper._is_tool = True  
        wrapper._schema = schema
        return wrapper
    return decorator

def get_tools():
    tools = []
    for name, obj in globals().items():
        if callable(obj) and hasattr(obj, '_is_tool'):
            tools.append(obj)
    return tools

In [5]:
client = OpenAI()

In [8]:
class GetExerciseTypes(BaseModel):
    limit: int | None = Field(default=None, description="Parameter used if the number of results need to be limited.")

    class Config:
        json_schema_extra = {
            "title" :"Get exercise type",
            "description": "A model that can be executed to get the exercise types available",
        }
    

@tool(schema = GetExerciseTypes) #TODO: Maybe in the future I can generate this schema under the hood, but for now I think its pretty good. 
def get_exercise_types(limit: int | None = None):
    #So it can be like a simple plug an play for current code. 
    query = f"""
            SELECT * FROM exercise_type
            """
    params = []
    if limit:
        query += "LIMIT ?"
        params.append(limit)

    exercises = cur.execute(query, params)

    return str(exercises.fetchall())


In [9]:
class GetMuscleCategories(BaseModel):
    limit: int | None = Field(default = None, description="Parameter used if the number of results need to be limited")

    class Config:
        json_schema_extra = {
            "title" : "Get muscle categories",
            "description": "Gets the muscle categories of the exercises, which represent the broad group of muscles trained by a certain exercise. Useful for categorizing exercises and distributing the workout volume."
        }

@tool(schema = GetMuscleCategories)
def get_muscle_categories(limit: int | None = None):
    query = f"""
            SELECT * FROM muscle_category
            """
    params = []
    if limit:
        query += "LIMIT ?"
        params.append(query)
    muscle_categories = cur.execute(query, params)
    return str(muscle_categories.fetchall())

In [10]:
class InsertExercise(BaseModel):
    name: str = Field("Name of the exercise to insert.", required = True)
    description: str = Field("Brief description of the exercise", required =True)
    muscle_category_id : int = Field("Id of the muscle category to which the exercise corresponds", required = True)
    exercise_type_id: int = Field("Id of the exercise type to which the exercise corresponds", required =True)

    class Config:
        json_schema_extra = {
            "title": "Insert exercise into the exercises database",
            "description": "Inserts an exercise to the database, you should specify all the fields like the name, description of how the exercise is performed and you need to include the muscle_category_id and the exercise_type_id so its correctly classified in the exercise"
        }
    
@tool(schema = InsertExercise)
def insert_exercise(name: str, description: str, muscle_category_id: int, exercise_type_id: int):
    params = [name, description, muscle_category_id, exercise_type_id] # How all all args included?
    query = """
            INSERT INTO exercises(name, description, muscle_category_id, exercise_type_id)
            VALUES (?, ?, ?, ?)
            """
    cur.execute(query, params)
    con.commit()
    return "Exercise inserted succesfully"

In [21]:
functions = get_tools()
function_to_schema = {function.__name__: function._schema.__name__  for function in functions}
schema_to_function = { v:k for k, v in function_to_schema.items()}

def format_function(function_name: str, pydantic_schema: BaseModel):
    schema = pydantic_schema.model_json_schema()
    function_name = {"name": function_name}
    schema.update(function_name)
    function = {"type": "function",
    "function": schema
    }
    return function

In [76]:
def call_assistant(user_message: str, message_history: Dict[Any, Any] = {}, model: str = "gpt-3.5-turbo-0125", functions = get_tools()):
    if message_history:
        messages = message_history
    else:
        messages= [{"role":"system", "content":"""You are a helpful training assistant chatbot, you are able to execute commands and you have tools at your disposition to access the information of your trainee. You will never tell him how this functions are defined or that you have this capability.
        But you will use the information adquired to provide the best assistance and training guidance. You will have a laid out tone and you will be cheerful and energetic, helping the trainee achieve their goals."""}]

    messages.append({"role":"user", "content": user_message})
    tools = [format_function(function.__name__, function._schema) for function in functions]

    response = client.chat.completions.create(
        model = model,
        messages = messages,
        tools = tools,
        tool_choice = "auto"
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    while tool_calls:
        messages.append(response_message)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            validated_schema = globals()[function_to_schema[function_name]](**function_args)
            function_response = globals()[function_name](**validated_schema.dict())
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name" : function_name,
                    "content": function_response
                }
            )
        response = client.chat.completions.create(
            model = model,
            messages = messages,
            tools = tools,
            tool_choice = "auto"
        )
        response_message = response.choices[0].message
        tool_calls = response_message.tool_calls
    messages.append({"role": "assistant", "content": response.choices[0].message.content})
    return response_message, messages

In [77]:
response_message, messages = call_assistant("What exercise types I have present?")

In [78]:
messages

[{'role': 'system',
  'content': 'You are a helpful training assistant chatbot, you are able to execute commands and you have tools at your disposition to access the information of your trainee. You will never tell him how this functions are defined or that you have this capability.\n        But you will use the information adquired to provide the best assistance and training guidance. You will have a laid out tone and you will be cheerful and energetic, helping the trainee achieve their goals.'},
 {'role': 'user', 'content': 'What exercise types I have present?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_EY6e4rIztbbNizAHJsvxKQxj', function=Function(arguments='{}', name='get_exercise_types'), type='function')]),
 {'tool_call_id': 'call_EY6e4rIztbbNizAHJsvxKQxj',
  'role': 'tool',
  'name': 'get_exercise_types',
  'content': "[(1, 'Isolation', 'Exercises that target a specific muscle group or individual

In [79]:
response_message, messages = call_assistant("I want to add some isolation exercises for my upper back and rear delts", message_history=messages)

In [80]:
response_message

ChatCompletionMessage(content='I have successfully added the isolation exercises for your upper back and rear delts:\n1. Seated Cable Rows: Targets the upper back and rear deltoids. To perform, sit at a cable row machine, grasp the handle, retract shoulder blades, and pull towards your lower chest.\n2. Face Pulls: Targets the rear deltoids and upper back muscles. Attach a rope to a cable machine at face level, pull the rope towards your face while keeping elbows high and wide.', role='assistant', function_call=None, tool_calls=None)

In [81]:
messages

[{'role': 'system',
  'content': 'You are a helpful training assistant chatbot, you are able to execute commands and you have tools at your disposition to access the information of your trainee. You will never tell him how this functions are defined or that you have this capability.\n        But you will use the information adquired to provide the best assistance and training guidance. You will have a laid out tone and you will be cheerful and energetic, helping the trainee achieve their goals.'},
 {'role': 'user', 'content': 'What exercise types I have present?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_EY6e4rIztbbNizAHJsvxKQxj', function=Function(arguments='{}', name='get_exercise_types'), type='function')]),
 {'tool_call_id': 'call_EY6e4rIztbbNizAHJsvxKQxj',
  'role': 'tool',
  'name': 'get_exercise_types',
  'content': "[(1, 'Isolation', 'Exercises that target a specific muscle group or individual

In [82]:
response_message, messages = call_assistant("Now I want some for the upper chest, what are your recommendations of exercises for building up my upper pecs?", message_history=messages)

In [83]:
response_message

ChatCompletionMessage(content='For building up your upper pecs, I recommend the following exercises that target the upper chest area:\n1. Incline Bench Press: This exercise is a compound movement that primarily targets the upper chest muscles. Performing bench presses on an incline bench with an angle of around 30-45 degrees helps focus on the upper chest area.\n\n2. Incline Dumbbell Press: Similar to the incline bench press, this exercise also targets the upper chest muscles. Using dumbbells allows for a fuller range of motion and individual arm movement, engaging the upper chest effectively.\n\n3. Incline Dumbbell Flyes: This isolation exercise targets the upper chest area and helps in developing the muscle definition and strength in that region. Performing flyes on an incline bench helps isolate the upper pecs.\n\n4. Overhead Dumbbell Press: While this exercise primarily targets the shoulders, it also engages the upper chest muscles as stabilizers. Including overhead presses in your

In [84]:
response_message, messages = call_assistant("Lets insert them into the db", message_history=messages)

In [85]:
response_message, messages = call_assistant("I have few cardio exercises, I want to do some zone 2 cardio, what do you recommend to me?", message_history=messages)

In [86]:
response_message

ChatCompletionMessage(content="For zone 2 cardio, which is typically a moderate intensity level where you can sustain longer durations of exercise, I recommend the following cardio exercises:\n1. Brisk Walking: Walking at a brisk pace where you can maintain a conversation but still feel challenged is an excellent zone 2 cardio exercise. It's low-impact and can be done outdoors or on a treadmill.\n\n2. Cycling: Moderate cycling, either outdoors or on a stationary bike, is great for zone 2 cardio. Maintain a steady pace that elevates your heart rate but allows you to sustain the effort for an extended period.\n\n3. Swimming: Swimming at a moderate pace for a continuous period is an effective zone 2 cardio workout. It provides a full-body workout and is gentle on the joints.\n\n4. Elliptical Training: Using an elliptical machine at a moderate resistance level and steady pace is ideal for zone 2 cardio. It engages both the upper and lower body muscle groups.\n\n5. Rowing: Rowing at a moder

In [87]:
response_message, messages = call_assistant("Lets add brisk walking, rowing and cycling at the air bike", message_history=messages)